In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os ; import re

In [87]:
npnts = 10001
nvib = 36
nJ = 81

#-------------------------------------------------------------------------------#
# EXTRAER AUTOFUNCIONES VIBRACIONALES
#-------------------------------------------------------------------------------#
def is_head_eigenvib(line:str) -> bool:
    """
    Detecta la linea que marca el comienzo de 
    una nueva autofunción de un nivel vibracional
    Formato: 1 0.0000 1 0 X 3Sigmam -> 
    -------> Indice Energia Estado NVib Nombre Estado
    """

    s = line.strip()
    if not s:
        return False
    
    parts = s.split()
    #Si no tiene más de 2 campos
    if len(parts) < 2:
        return False
    
    #POR SI ACASOS
    #Si el primer campo no es un entero
    if not re.fullmatch(r"[+-]?\d+", parts[0]):
        return False
    
    #Si el segundo campo no es un valor decimal
    if not re.fullmatch(r"[+-]?(\d+(\.\d*)?|\.\d+)([Ee][+-]?\d+)?", parts[1]):
        return False
    
    return True

#-------------------------------------------------------------------------------#
#Parsing Vibrational Eigenfunctions
#-------------------------------------------------------------------------------#

def parse_duo_vib_einfun(fname:str, npoints: int, nvib: int):
    """
    Parsea el archivo de autofunciones de vibración de DUO
    """
    f = open(fname, 'r')
    lines = f.readlines()

    vibmat = np.zeros((npoints,nvib))
    evect = np.zeros((nvib))
    ncurrpoint = 0 ; ncurrvib = 0
    for i,line in enumerate(lines):
        if is_head_eigenvib(line):
            ncurrpoint = 0
            ncurrvib = int(line.split()[3])
            evect[ncurrvib] = float(line.split()[1])
        elif "End of contracted basis" in line:
            break
        else:
            vibmat[ncurrpoint,ncurrvib] = float(line)
            ncurrpoint += 1

    return vibmat

#-------------------------------------------------------------------------------#
# Reading Dyson Matrix
#-------------------------------------------------------------------------------#

def dyson_mat(fname:str):
    dysvals = np.loadtxt(fname, dtype=np.float64)
    rvals = dysvals[:,0]; dys_N_C1 = dysvals[:,1] ; dys_N_C2 = dysvals[:,2]
    dysmat_N_C1 = np.zeros((dysvals.shape[0],dysvals.shape[0]))
    dysmat_N_C2 = np.zeros((dysvals.shape[0],dysvals.shape[0]))

    for i in range(dysvals.shape[0]):
            dysmat_N_C1[i][i] = dys_N_C1[i]*0.393456
            dysmat_N_C2[i][i] = dys_N_C2[i]*0.393456

    return rvals,dysmat_N_C1, dysmat_N_C2

def bra_ket(vib_ini, vib_fin, dymat):
    
    intensity = (vib_fin.T @ (dymat @ vib_ini))
    return intensity

#-------------------------------------------------------------------------------#
# Reading Energy as a function of J
#-------------------------------------------------------------------------------#

def getJvals(fname:str, nvib: int, nj: int, mult: int):
    f = open(fname, 'r')
    lines = f.readlines()

    jn = 0
    for i,line in enumerate(lines):
        if i == 0:
            Lambda_abs = abs(int(float(line.split()[5])))
            if Lambda_abs == 0:
                nOmega = mult
            else:
                nOmega = (Lambda_abs+1) * mult
            Jvals = np.zeros((nvib,nj,nOmega))

        parts = line.split()
        vibn = int(parts[4])
        jn = int(round(float(parts[0])))

        #Getting indexes for J projections
        S=(mult-1)/2
        mval = float(parts[8])
        midx = int(round(mval+S))
        
        if vibn < nvib:
            Jvals[vibn,jn,midx] = float(parts[2])
        else:
            continue

    return Jvals

#-------------------------------------------------------------------------------#

pathor="/home/jorgebdelafuente/Doctorado/Photoion/DUO/PHPHM/"
PHvibs = parse_duo_vib_einfun(pathor+"PH/vibeigenvect_vib.chk", npnts, nvib)
PHMGSvibs = parse_duo_vib_einfun(pathor+"PHM_GS/vibeigenvect_vib.chk", npnts, nvib)
PHMa4Smvibs = parse_duo_vib_einfun(pathor+"PHM_a4Sm/vibeigenvect_vib.chk", npnts, nvib)

PHJener = getJvals(pathor+"PH/rovibronic_energies.dat", nvib, nJ, 3)
PHMGSJener = getJvals(pathor+"PHM_GS/rovibronic_energies.dat", nvib, nJ, 2)
PHMa4SmJener = getJvals(pathor+"PHM_a4Sm/rovibronic_energies.dat", nvib, nJ, 4)

rvals,dysmat_N_C1,dysmat_N_C2 = dyson_mat(pathor+"Dipole_moment_functions.dat")
mask = (rvals > 0.793755) & (rvals < 1.8)
rvals = rvals[mask]
dysmat_N_C1 = dysmat_N_C1[mask] ; dysmat_N_C1 = dysmat_N_C1[:,mask]
dysmat_N_C2 = dysmat_N_C2[mask] ; dysmat_N_C2 = dysmat_N_C2[:,mask]
PHvibs_v0 = PHvibs[:,0] ; PHvibs_v0 = PHvibs_v0[mask]
PHMGSvibs_v0 = PHMGSvibs[:,0] ; PHMGSvibs_v0 = PHMGSvibs_v0[mask]
PHMa4Smvibs_v0 = PHMa4Smvibs[:,0] ; PHMa4Smvibs_v0 = PHMa4Smvibs_v0[mask]